# Text Classification - Training a GNN


## $\color{blue}{Sections:}$

* Preamble
1.   Admin
2.   Dataset
3.   Model
4.   Train - Validate
5.   Training Loop

## $\color{blue}{Preamble:}$

We now train a GNN in basic PyTorch. The model will look like a GCN. Inference willbe completed in another notebook as the whole graph must be uploaded at once.

## $\color{blue}{Admin}$
* Install relevant Libraries
* Import relevant Libraries

In [ ]:
import torch
import pandas as pd
from google.colab import drive
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


## $\color{blue}{Data}$

* Connect to Drive
* Load the data
* Load adjacency matrices

In [ ]:
path = 'class/datasets/'
df_train = pd.read_pickle(path + 'df_train')
df_dev = pd.read_pickle(path + 'df_dev')
df_test = pd.read_pickle(path + 'df_test')

In [ ]:
path = 'class/tensors/adj_{}.pt'

# train
train_people = torch.load(path.format('train_people'))
train_locations = torch.load(path.format('train_locations'))
train_entities = torch.load(path.format('train_entities'))

# dev
dev_people = torch.load(path.format('dev_people'))
dev_locations = torch.load(path.format('dev_locations'))
dev_entities = torch.load(path.format('dev_entities'))

# val (contains the adjacency matrix for both the training and the development set) #### train set stacked on top of dev set in the adjacency matrix
val_people = torch.load(path.format('val_people.1'))
val_locations = torch.load(path.format('val_locations.1'))
val_entities = torch.load(path.format('val_entities.1'))

<ipython-input-5-6f8839527fd4>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_people = torch.load(path.format('train_people'))
<ipython-input-5-6f8839527fd4>:5: Futu

## $\color{blue}{Sampling:}$

In [ ]:
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy

def sample_neighborhood(primary_inds, input, adj, distance, neighbor_max = 4):
    """
    Takes the given inds and the inputs, returns the sampled set of indices and the corresponding activation flag.
    If the activation flag is True, then the the datapoint is primary and has been search for neighbors and neighbors
    of neighbors. This indicates that we rely on the datapoint for metrics and loss.

    Args:
      primary_inds : iterable
          indices sampled by the dataloader
      input : torch.Tensor
          (m,d) input tensor
      adj : torch.Tensor
          (m,m) adjacency matrix
      distance : torch.Tensor
          (m,m) : cosine similarity between all inputs
      neighbour_max : int (optional)
          The maximum number of neighbors to consider for each point

    Returns:
      sampled_indices : list
          indices of all datapoints to be processed in the batch
      activation_flag : list
          boolean_flag indicating whether the corresponding datapoint is to be considered for metrics
    """

    def _get_closest_neighbors(ind):
      """get up to neighbor_max close neighbors"""
      local_neighbors = []
      local_activation_flag = []
      candidate_neighbors = [neighbor.item() for neighbor in (adj[ind] > 0).nonzero(as_tuple=True)[0] if neighbor.item() not in sampled_indices]
      candidate_distances = [(neighbor, distance[primary_ind][neighbor]) for neighbor in candidate_neighbors]
      sorted_neighbors = sorted(candidate_distances, key=lambda x: x[1])
      return [neighbor for neighbor, dist in sorted_neighbors[:neighbor_max]], candidate_neighbors

    sampled_indices = []
    activation_flag = []
    all_banned_neighbors = []

    for primary_ind in primary_inds:

      # if primary ind has been added as a neighbor, convert the activation flag to true, else add it as a standard primary index
      if primary_ind in sampled_indices:
        activation_flag[sampled_indices.index(primary_ind)] = True
      else:
        sampled_indices.append(primary_ind)
        activation_flag.append(True)

      # print('\n', primary_ind)
      # print('sampled_indices', sampled_indices)

      level_1_neighbors, candidate_neighbors = _get_closest_neighbors(primary_ind)
      banned_neighbors = list(set(candidate_neighbors) - set(level_1_neighbors))
      all_banned_neighbors.extend(banned_neighbors)
      # print('banned_neighbors', banned_neighbors)
      # print('all_banned_neighbors', all_banned_neighbors)

      level_1_activation_flag = [False for el in level_1_neighbors]
      sampled_indices.extend(level_1_neighbors)
      activation_flag.extend(level_1_activation_flag)

      # print('level_1_neighbors', level_1_neighbors)
      # print('sampled_indices', sampled_indices)
      # print('level_1_activation_flag', level_1_activation_flag)
      # print('activation_flag', activation_flag)

      for level_1_ind in level_1_neighbors:
        level_2_neighbors, _ = _get_closest_neighbors(level_1_ind)
        level_2_activation_flag = [False for el in level_2_neighbors]
        sampled_indices.extend(level_2_neighbors)
        activation_flag.extend(level_2_activation_flag)

        # print('level_2_neighbors', level_2_neighbors)
        # print('sampled_indices', sampled_indices)
        # print('level_2_activation_flag', level_2_activation_flag)
        # print('activation_flag', activation_flag)

    # include only 4 level one neighbors for each primary index to avoid pollution
    clean_indices = []
    clean_flags = []
    for i in range(len(sampled_indices)):
      target = sampled_indices[i]
      flag = activation_flag[i]
      if target in primary_inds:
        clean_indices.append(target)
        clean_flags.append(flag)
      elif target in all_banned_neighbors:
        continue
      else:
        clean_indices.append(target)
        clean_flags.append(flag)
    # print(f'\ncleaning\nsampled_indices : {sampled_indices}\nclean_indices : {clean_indices}')
    return clean_indices, clean_flags


## $\color{blue}{Sampling Test:}$

In [ ]:
import torch
import numpy as np
a = torch.Tensor([
    [0,1,0,0,0,0,0,1],
    [1,0,0,1,0,1,0,0],
    [0,1,0,1,0,0,1,0],
    [0,1,0,0,1,0,0,1],
    [1,1,0,0,1,0,0,0],
    [0,0,1,1,0,1,1,0],
    [0,0,0,1,0,0,0,1],
    [0,0,0,1,0,1,1,1]
])

seed=42

In [ ]:
import torch

def neighbor_analysis(primary_inds: list[int], inds: list[int], adjacency_matrix: torch.Tensor):
    result = []

    # Loop over each primary index
    for primary_idx in primary_inds:
        # Get neighbors for the primary index
        neighbors = []
        for ind in inds:
            # Check if ind is a neighbor of primary_idx
            if adjacency_matrix[primary_idx, ind] == 1:
                neighbors.append(ind)

        # Find neighbors of neighbors (n + 1 neighbors)
        neighbors_of_neighbors = []
        for neighbor in neighbors:
            for ind in inds:
                # Check if ind is a neighbor of the current neighbor
                if adjacency_matrix[neighbor, ind] == 1 and ind != primary_idx:
                    neighbors_of_neighbors.append(ind)

        # Remove duplicates for the neighbors of neighbors
        neighbors_of_neighbors = list(set(neighbors_of_neighbors))

        # Add the tuple to results
        result.append((primary_idx, neighbors, neighbors_of_neighbors))

    return result

primary_inds, input, adj, distance,

In [ ]:
def calculate_cosine(H):
    dot_product = torch.matmul(H, H.transpose(0, 1))  # shape (m, m)

    lengths = torch.sqrt(dot_product.diagonal()).unsqueeze(1)  # shape (m, 1)
    denominator = lengths @ lengths.transpose(0, 1)  # shape (m, m)

    return dot_product / denominator  # shape (m, m)

In [ ]:
H = torch.stack(list(df_train['vanilla_embedding.1']))


In [ ]:
distance = calculate_cosine(H)

In [ ]:
import random
inds = random.sample(list(range(12000)),6)
print(inds)
sample = sample_neighborhood(inds,H,train_entities,distance)
print(sample)
print(len(sample[0]))

[6339, 2165, 4826, 11292, 11280, 5401]

 6339
sampled_indices [6339]
banned_neighbors [9728, 3589, 7176, 11273, 11784, 3084, 3597, 5644, 10259, 11796, 2581, 5653, 6170, 3617, 6690, 7203, 11302, 552, 8755, 10804, 6198, 3640, 8252, 2622, 64, 7744, 10305, 10817, 10823, 9802, 10826, 591, 81, 7250, 5207, 4187, 9821, 10847, 2144, 9825, 11872, 4195, 11877, 5223, 4717, 10861, 8305, 5246, 9342, 1664, 5760, 3714, 10879, 6276, 10375, 648, 11915, 144, 2195, 3733, 10393, 6298, 6814, 8865, 6819, 1702, 9897, 4781, 8365, 1711, 3762, 7346, 9906, 7864, 11449, 2234, 5307, 7866, 3261, 11453, 8383, 10431, 1730, 3266, 2247, 4808, 9415, 6862, 2768, 3797, 9431, 6363, 7387, 10462, 5856, 7394, 1253, 6377, 7403, 2796, 2286, 3311, 1776, 5876, 7928, 2810, 765, 5885, 9471, 1794, 1795, 6403, 6920, 1290, 2315, 4363, 1293, 8464, 11025, 8468, 3353, 10532, 8485, 7975, 9511, 10024, 1838, 3375, 306, 11571, 6965, 7477, 9525, 7482, 2366, 4414, 2880, 3392, 8000, 4419, 8001, 11595, 7501, 10576, 9042, 4436, 345, 3418, 9568, 24

In [ ]:
analysis = neighbor_analysis(inds, sample[0], train_entities)
for item in analysis:
  print(item)

(6339, [10087, 11957, 10768, 11320], [8961, 10768, 26, 6427, 8605, 6948, 167, 11957, 11320, 8133, 1226, 11083, 721, 8020, 6232, 3420, 10461, 2143, 4576, 10087, 11625, 6129, 7409, 2803, 2165])
(2165, [10087, 6427, 8605, 4576, 8020], [8961, 10768, 26, 6427, 8605, 167, 11957, 11320, 6339, 8133, 11083, 721, 8020, 6232, 3420, 4576, 10087, 11625, 6129, 7409, 2803])
(4826, [], [])
(11292, [6129, 2143, 9235, 8778, 11085, 3998], [8961, 2183, 6283, 2701, 655, 10768, 1936, 8465, 9235, 6427, 3613, 4894, 3998, 545, 2341, 167, 2425, 4905, 5419, 560, 8114, 9910, 11320, 4028, 8778, 1612, 11085, 721, 3420, 4574, 2143, 6116, 613, 10087, 3815, 11625, 7399, 7409, 7153, 2803, 5108, 6129, 10489, 1274])
(11280, [], [])
(5401, [9910, 5054, 1274], [2183, 6283, 2701, 655, 1936, 9235, 4894, 545, 2341, 5419, 560, 11186, 4028, 5054, 8778, 1612, 4685, 4695, 4574, 3815, 7399, 8812, 6129, 7153, 5108, 10489, 1274])


In [ ]:
# inds = random.sample(list(range(12000)),4)
inds = list(range(101,105))
sample = sample_neighborhood([train_entities], inds, 16, 64, seed=42)
print(sample)
print(len(sample))

[9221, 2061, 3599, 10261, 9214, 6937, 4386, 2855, 8492, 4654, 10039, 4152, 2363, 11581, 317, 6210, 580, 6725, 9046, 599, 5721, 3673, 1883, 3679, 4707, 4964, 101, 102, 103, 104, 8551, 5480, 5485, 11121, 3953, 7285, 8824, 11904, 6274, 9607, 11660, 7317, 5271, 8856, 3235, 7594, 4524, 5550, 10927, 4023, 1722, 4030, 10430, 8641, 4305, 8916, 9688, 2009, 10203, 10207, 9440, 9185, 1250, 8680, 6640, 11763, 2804, 510]
68


## $\color{blue}{Dataset:}$

In [ ]:
import torch

class GNNDataset(Dataset):
  def __init__(self, H, A, labels, length, neighbor_max=4):
    """Custom dataset with neighborhood sampling

    Args:
      H : torch.tensor
        input embeddings (n x d)

      A : torch.tensor
        adjacency matrix (n x n)

      labels : torch.LongTensor
        y

      meta_indices : torch.LongTensor
        index of datapoint to filter validation score

      neighbor_max : int
        max neighbors for each node in mini-batch

      batch_max : int
        max size of batch

    """
    # All inits must be tensors
    self.H = H.to(device)
    self.A = A.to(device)
    self.cosine = self.calculate_cosine(self.H)
    self.labels = labels.to(device)
    self.neighbor_max = neighbor_max
    self.length = length

  def __len__(self):
    return self.length

  def __getitem__(self, inds):
    # Sample neighborhood

    # get inds in list
    inds = inds.tolist() if torch.is_tensor(inds) else (inds if isinstance(inds,list) else [inds])

    # return the required inds (The inds are the sampes and the active flag dictates relatively if that sample should be counted)
    sampled_indices, active_flag = sample_neighborhood(inds, self.H, self.A, self.cosine, self.neighbor_max)

    # get the input for the required inds
    H_batch = self.H[sampled_indices]

    # get the adjacency matrix for the required inds
    A_batch = self.A[sampled_indices][:, sampled_indices]

    # get the labels for the required inds
    labels_batch = self.labels[sampled_indices]

    return H_batch, A_batch, labels_batch, torch.LongTensor(active_flag).to(device)

  def calculate_cosine(self, H):
      dot_product = torch.matmul(H, H.transpose(0, 1))  # shape (m, m)

      lengths = torch.sqrt(dot_product.diagonal()).unsqueeze(1)  # shape (m, 1)
      denominator = lengths @ lengths.transpose(0, 1)  # shape (m, m)

      return dot_product / denominator  # shape (m, m)


 H, A, labels, length, neighbor_max=4

In [ ]:
# training loader
H_train = torch.stack(list(df_train['vanilla_embedding.1']))
labels_train = torch.LongTensor(list(df_train['chapter_idx']))
A_train = train_entities


train_dataset = GNNDataset(H_train, A_train, labels_train, H_train.size(0), neighbor_max=4)

# Prevent dataloader from calling a single index at a time
custom_train_sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.RandomSampler(train_dataset),
    batch_size=8,
    drop_last=False)


train_loader = DataLoader(train_dataset, sampler = custom_train_sampler)

# validation loader
df1 = df_train[['vanilla_embedding.1', 'chapter_idx']]
df2 = df_dev[['vanilla_embedding.1', 'chapter_idx']]
df_val = pd.concat([df2, df1])
H_val = torch.stack(list(df_val['vanilla_embedding.1']))
labels_val = torch.LongTensor(list(df_val['chapter_idx']))
A_val = val_entities

validation_dataset = GNNDataset(H_val, A_val, labels_val, df2.shape[0], neighbor_max=4)

# Prevent dataloader from calling a single index at a time
custom_validation_sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.SequentialSampler(validation_dataset),
    batch_size=1024,
    drop_last=False)

dev_loader = DataLoader(validation_dataset, sampler = custom_validation_sampler)


In [ ]:
# Check batches

# Number of batches to inspect
num_batches_to_check = 2

for batch_idx, (inputs, adjacency, labels, flag) in enumerate(train_loader):
    print('\n##########################\n')
    print(f"Batch {batch_idx + 1}/{num_batches_to_check}:")
    print('-' * 10)
    print("Inputs:")
    print(f"  Type: {type(inputs)}")
    print(f"  Shape: {inputs.size()}")
    print('-' * 10)
    print("Adjacency:")
    print(f"  Type: {type(adjacency)}")
    print(f"  Shape: {adjacency[0].size()}")
    print('-' * 10)
    print("Indices:")
    print(f"  Type: {type(flag)}")
    print(f"  Shape: {flag.size()}")
    print(flag)
    print('-' * 10)
    print("Labels:")
    print(f"  Type: {type(labels)}")
    print(f"  Shape: {labels.size()}")
    print(labels)

    # Stop after inspecting the desired number of batches
    if batch_idx + 1 >= num_batches_to_check:
        break


##########################

Batch 1/2:
----------
Inputs:
  Type: <class 'torch.Tensor'>
  Shape: torch.Size([1, 52, 768])
----------
Adjacency:
  Type: <class 'torch.Tensor'>
  Shape: torch.Size([52, 52])
----------
Indices:
  Type: <class 'torch.Tensor'>
  Shape: torch.Size([1, 52])
tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0]], device='cuda:0')
----------
Labels:
  Type: <class 'torch.Tensor'>
  Shape: torch.Size([1, 52])
tensor([[64, 25, 25, 25, 25, 25, 16, 40, 33, 16, 17, 57, 16, 16, 16, 16, 16, 14,
         25, 25, 25, 25, 68, 15, 62, 42, 58, 49, 47, 44, 58, 56, 37, 49, 50, 41,
         48, 50, 17, 42, 12, 46, 63, 40, 40, 35, 39, 58,  8, 47, 58, 45]],
       device='cuda:0')

##########################

Batch 2/2:
----------
Inputs:
  Type: <class 'torch.Tensor'>
  Shape: torch.Size([1, 62, 768])
----------
Adjacency:
  Type: <class 'torch.Ten

The dataloader seems to be working correctly, the implementation of custom sampling on all indices at once leads to DataLoaders collate function inserting a new dimension that it will stach against. Because all indices are dealt with at once, there is no stacking.

The simple solution will be to simply squeeze the tensors in the training loop. The validation loader eradicates randomness from the process.

## $\color{blue}{Model:}$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.5, training=True):
        super(GNNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_relations = num_relations
        self.dropout = dropout
        self.training = training

        self.T = nn.Parameter(torch.Tensor(in_features, out_features))
        self.E = nn.Parameter(torch.Tensor(in_features, out_features))

        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(out_features)

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.T)
        nn.init.xavier_uniform_(self.E)

    def forward(self, H, A):
        messages_projection = A.T @ H @ self.E
        degrees = A.sum(dim=1, keepdim=True)
        degrees[degrees == 0] = 1.0
        messages_projection /= degrees

        self_projection = H @ self.T

        # Include skip connection
        H_out = F.leaky_relu(self_projection + messages_projection) + H
        H_out = F.dropout(H_out, p=self.dropout)

        # Apply batch normalization
        H_out = self.batch_norm(H_out)

        return H_out

class GNNModel(nn.Module):
    def __init__(self, d, h, c, num_layers=2, dropout=0.5):
        super(GNNModel, self).__init__()
        self.num_layers = num_layers
        self.gnn_layers = nn.ModuleList([GNNLayer(d, d, dropout) for _ in range(num_layers)])
        self.fc1 = nn.Linear(d, h)
        self.batch_norm_fc1 = nn.BatchNorm1d(h)
        self.fc2 = nn.Linear(h, c)
        self.dropout = dropout
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, H, A):
        for layer in self.gnn_layers:
            H = layer(H, A)

        H = F.dropout(H, p=self.dropout, training=self.training)
        H = F.relu(self.batch_norm_fc1(self.fc1(H)))
        Output = self.fc2(H)
        return Output

    def forward_layer(self, H, A, layer_idx):
        """Forward pass for a specific layer."""
        H = self.gnn_layers[layer_idx](H, A)
        return H


In [ ]:
import torch.optim as optim

d = 768
h = 400   # hidden dimension of fully connected layer
c = 70   # number of classes
num_relations = 1   # number of relationship types

# Model, Loss, Optimizer
model = GNNModel(d, h, c)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)



In [ ]:
def count_parameters_per_module(model):
    print("Module and parameter counts:")

    for name, module in model.named_modules():
        # Skip the top-level module (the model itself)
        if not isinstance(module, nn.Module) or name == "":
            continue

        param_count = sum(p.numel() for p in module.parameters() if p.requires_grad)

        if param_count > 0:  # Only print modules that have parameters
            print(f"{name}: {param_count} parameters")

In [ ]:
count_parameters_per_module(model)

Module and parameter counts:
gnn_layers: 2362368 parameters
gnn_layers.0: 1181184 parameters
gnn_layers.0.batch_norm: 1536 parameters
gnn_layers.1: 1181184 parameters
gnn_layers.1.batch_norm: 1536 parameters
fc1: 307600 parameters
batch_norm_fc1: 800 parameters
fc2: 28070 parameters


## $\color{blue}{Train-Validate:}$

In [ ]:
def accuracy(outputs, labels):
    # argmax to get predicted classes
    _, predicted = torch.max(outputs, 1)

    # count correct
    correct = (predicted == labels).sum().item()

    # get average
    acc = correct / labels.size(0)  # Total number of samples
    return acc

In [ ]:
import numpy as np

def train(model, train_loader, criterion, optimizer):
    model.train()
    epoch_train_losses = []
    epoch_train_accuracy = []

    for batch_idx, (H, A, y, flag) in enumerate(train_loader):
        optimizer.zero_grad()

        H = H.squeeze(0)
        A = A.squeeze(0)
        y = y.squeeze(0)
        flag = flag.squeeze(0).bool()

        out = model(H,A)

        filtered_out = out[flag]
        filtered_y = y[flag]

        train_loss = criterion(filtered_out, filtered_y)
        train_accuracy = accuracy(filtered_out, filtered_y)


        epoch_train_losses.append(train_loss.item())
        epoch_train_accuracy.append(train_accuracy)

        # Backpropagation and optimization
        train_loss.backward()
        optimizer.step()

    return np.mean(epoch_train_losses), np.mean(epoch_train_accuracy)

In [ ]:
def validate(model, dev_loader, criterion, threshold=12000):
    model.eval()
    epoch_dev_losses = []
    epoch_dev_accuracy = []
    pred_holder = []
    real_holder = []

    with torch.no_grad():
        for batch_idx, (H, A, y, flag) in enumerate(dev_loader):
            H = H.squeeze(0)
            A = A.squeeze(0)
            y = y.squeeze(0)
            flag = flag.squeeze(0).bool()

            out = model(H, A)

            filtered_out = out[flag]
            filtered_y = y[flag]


            dev_loss = criterion(filtered_out, filtered_y)
            dev_accuracy = accuracy(filtered_out, filtered_y)

            epoch_dev_losses.append(dev_loss.item())
            epoch_dev_accuracy.append(dev_accuracy)

    # Avoid division by zero if no validation points were processed
    return np.mean(epoch_dev_losses), np.mean(epoch_dev_accuracy)

## $\color{blue}{Training:}$

In [ ]:
from collections import namedtuple
Stats = namedtuple('Stats', [
    'train_loss',
    'train_accuracy',
    'dev_loss',
    'dev_accuracy',
    'epoch',
    'lr',
    'alpha',
    'max_accuracy'
])

In [ ]:
def gen_config(lr_low, lr_high, alpha_low, alpha_high):
  np.random.seed()
  lr = round(10**float(np.random.uniform(lr_low,lr_high)),6)
  alpha = round(10**float(np.random.uniform(alpha_low,alpha_high)),6)
  return lr, alpha

In [ ]:
def gen_ranges( lr, lr_range, alpha, alpha_range):

  lr_center = lr
  lr_low = lr_center - lr_range/2
  lr_high = lr_center + lr_range/2
  lr_diff = lr_high - lr_low

  alpha_center = alpha
  alpha_low = alpha_center - alpha_range/2
  alpha_high = alpha_center + alpha_range/2
  alpha_diff = alpha_high - alpha_low

  return (lr_low, lr_high, alpha_low, alpha_high)

In [ ]:
def search_stats(results):
  best_stats = None
  max_dev_accuracy = 0
  for i in range(len(results)):
    acc = results[i].dev_accuracy
    if acc > max_dev_accuracy:
      best_stats = results[i]
      max_dev_accuracy = acc
  return best_stats

In [ ]:
def tv_run(epochs, model, lr, alpha, max_accuracy, path, verbose = 0):
  """
  Runs a training setup
  verbose == 1 - print model results
  verbose == 2 -> print epoch and model results
  """
  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=alpha)

  # Prepare data loaders
  train_loader = DataLoader(train_dataset, sampler = custom_train_sampler)
  dev_loader = DataLoader(validation_dataset, sampler = custom_validation_sampler)

  # Hold epoch stats
  train_losses = []
  train_accuracy = []
  dev_losses = []
  dev_accuracy = []
  epoch_holder = []

  # Break if no improvement
  current_best = 0
  no_improvement = 0


  # Run epochs
  for epoch in range(epochs):

    # break out of epochs
    if no_improvement >= 6:
      break

    # call training and validation functions
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    dev_loss, dev_acc = validate(model, dev_loader, criterion)

    # Store epoch stats
    train_losses.append(train_loss)
    train_accuracy.append(train_acc)
    dev_losses.append(dev_loss)
    dev_accuracy.append(dev_acc)
    epoch_holder.append(epoch + 1)

    # check for improvement
    if dev_acc > current_best:
      current_best = dev_acc
      no_improvement = 0
    else:
      no_improvement += 1

    # save best model
    if dev_acc > max_accuracy:
      torch.save(model.state_dict(), path)
      max_accuracy = dev_acc


    # optionally print epoch results
    if verbose == 2:
      print(f'\n --------- \nEpoch: {epoch + 1}\n')
      print(f'Epoch {epoch + 1} train loss: {train_loss:.4f}')
      print(f'Epoch {epoch + 1} train accuracy: {train_acc:.4f}')
      print(f'Epoch {epoch + 1} dev loss: {dev_loss:.4f}')
      print(f'Epoch {epoch + 1} dev accuracy: {dev_acc:.4f}')

      # save best results
  max_ind = np.argmax(dev_accuracy)

  stats = Stats(
      train_losses[max_ind],
      train_accuracy[max_ind],
      dev_losses[max_ind],
      dev_accuracy[max_ind],
      epoch_holder[max_ind],
      lr, alpha,
      max_accuracy
  )

  # optionally print model results
  if verbose in [1,2]:
    print('\n ######## \n')
    print(f'lr:{stats.lr}, alpha:{stats.alpha} @ epoch {stats.epoch}.')
    print(f'TL:{stats.train_loss}, TA:{stats.train_accuracy}.')
    print(f'DL:{stats.dev_loss}, DA:{stats.dev_accuracy}')

  return stats

#### $\color{red}{Sanity-check:}$

In [ ]:
# model
model = GNNModel(d, h, c)

In [ ]:
# training loader
H_train = torch.stack(list(df_train['vanilla_embedding.1']))
labels_train = torch.LongTensor(list(df_train['chapter_idx']))
A_train = train_entities


train_dataset = GNNDataset(H_train, A_train, labels_train, H_train.size(0), neighbor_max=4)

# Prevent dataloader from calling a single index at a time
custom_train_sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.RandomSampler(train_dataset),
    batch_size=256,
    drop_last=False)


train_loader = DataLoader(train_dataset, sampler = custom_train_sampler)

# validation loader
df1 = df_train[['vanilla_embedding.1', 'chapter_idx']]
df2 = df_dev[['vanilla_embedding.1', 'chapter_idx']]
df_val = pd.concat([df2, df1])
H_val = torch.stack(list(df_val['vanilla_embedding.1']))
labels_val = torch.LongTensor(list(df_val['chapter_idx']))
A_val = val_entities

validation_dataset = GNNDataset(H_val, A_val, labels_val, df2.shape[0], neighbor_max=4)

# Prevent dataloader from calling a single index at a time
custom_validation_sampler = torch.utils.data.sampler.BatchSampler(
    torch.utils.data.sampler.SequentialSampler(validation_dataset),
    batch_size=1024,
    drop_last=False)

dev_loader = DataLoader(validation_dataset, sampler = custom_validation_sampler)


In [ ]:
epochs = 2
lr = 0.0005
alpha = 0.0001
path = "class/models/GNN_distance_sampler.pt"
max_accuracy = 0
model = GNNModel

In [ ]:
tv_run(epochs, model, lr, alpha, max_accuracy, path, verbose = 2)

[3351, 2961, 1285, 11337, 10087, 9692, 10548, 1902]
[11313, 9937, 7837, 10536, 11687, 9238, 11239, 5075]
[10284, 6339, 4617, 7661, 2543, 3138, 11978, 11237]
[1364, 6227, 9042, 5224, 7459, 1499, 2438, 10970]
[3210, 7092, 10889, 7501, 536, 7731, 7564, 2785]
[5961, 6612, 8085, 7418, 10261, 10557, 10888, 5634]
[4770, 1478, 3988, 7703, 1954, 401, 3216, 6247]
[10301, 8860, 11203, 11517, 4248, 2303, 8128, 7034]
[6071, 3634, 3199, 2586, 8699, 6732, 6274, 6016]
[7061, 5872, 7977, 6859, 3872, 10797, 1767, 10002]
[1832, 2426, 8090, 4733, 741, 2342, 7554, 9251]
[1068, 8486, 1210, 1807, 8642, 7645, 3396, 3679]
[8188, 5558, 1521, 940, 10845, 10791, 5361, 10150]
[6437, 3148, 4963, 4060, 10691, 760, 9404, 8283]
[10943, 4561, 5300, 839, 118, 941, 820, 11949]
[3569, 10426, 3808, 10730, 2647, 11624, 10677, 9683]
[11919, 3391, 10789, 7591, 9429, 7432, 1208, 11011]
[6709, 2267, 4404, 3724, 2874, 9964, 3376, 6983]
[5099, 474, 3078, 11491, 3052, 4808, 4289, 3154]
[9043, 775, 10721, 781, 6209, 3440, 9473, 416

KeyboardInterrupt: 

#### $\color{red}{Run:}$

In [ ]:
"""
Main Admin
"""
epochs =
max_accuracy = 0
path = "class/models/GNN.4.pt"
results = []

"""
init random search
lr [10^-5 - 10^-1]
alpha [10^-5 - 10^-1]
bs [8, 32, 128]
"""
lr_low = -4
lr_high = -3
lr_range = lr_high - lr_low

alpha_low = -5
alpha_high = -3
alpha_range = alpha_high - alpha_low

d = 768
h = 400
c = 70

count = 0

"""
Hyperparameter Search
"""

for i in range(3):
  # debug
  print("\n################\n")
  print(f'round: {i}')
  # print(f'lr_low{lr_low}, lr_high{lr_high}, lr_range{lr_range}')
  # print(f'alpha_low{alpha_low}, lr_high{alpha_high}, lr_range{alpha_range}')
  print('max', max_accuracy)
  print("\n################\n")


  for j in range(4):
    count += 1
    print(count)

    # get config
    lr, alpha = gen_config(lr_low, lr_high, alpha_low, alpha_high)
    # define model
    model = GNNModel(d, h, c)
    model = model.to(device)

    # run training
    res = tv_run(epochs, model, lr, alpha, max_accuracy, path, verbose = 2)
    max_accuracy = res.max_accuracy
    results.append(res)

  # get best result of the round or even so far
  stats = search_stats(results)


  print(stats) # debug

  # reconfigure the new hypers
  lr = np.log10(stats.lr)
  lr_range = lr_range / 3

  alpha = np.log10(stats.alpha)
  alpha_range = alpha_range / 3

  config = gen_ranges(lr, lr_range, alpha, alpha_range)
  lr_low, lr_high, alpha_low, alpha_high = config
  lr_range = lr_high - lr_low
  alpha_range = alpha_high - alpha_low



################

round: 0
max 0

################

1

 --------- 
Epoch: 1

Epoch 1 train loss: 4.2382
Epoch 1 train accuracy: 0.0793
Epoch 1 dev loss: 3.9854
Epoch 1 dev accuracy: 0.1307

 --------- 
Epoch: 2

Epoch 2 train loss: 3.8650
Epoch 2 train accuracy: 0.1430
Epoch 2 dev loss: 3.6485
Epoch 2 dev accuracy: 0.1815

 --------- 
Epoch: 3

Epoch 3 train loss: 3.6622
Epoch 3 train accuracy: 0.1682
Epoch 3 dev loss: 3.4355
Epoch 3 dev accuracy: 0.2168

 --------- 
Epoch: 4

Epoch 4 train loss: 3.4748
Epoch 4 train accuracy: 0.1940
Epoch 4 dev loss: 3.2958
Epoch 4 dev accuracy: 0.2158

 --------- 
Epoch: 5

Epoch 5 train loss: 3.3187
Epoch 5 train accuracy: 0.2140
Epoch 5 dev loss: 3.1096
Epoch 5 dev accuracy: 0.2552

 --------- 
Epoch: 6

Epoch 6 train loss: 3.1599
Epoch 6 train accuracy: 0.2343
Epoch 6 dev loss: 3.0127
Epoch 6 dev accuracy: 0.2656

 --------- 
Epoch: 7

Epoch 7 train loss: 3.0315
Epoch 7 train accuracy: 0.2520
Epoch 7 dev loss: 2.8960
Epoch 7 dev accuracy: 0.2832


In [ ]:
alpha = 0.000046
lr = 0.000124
epochs = 30
model = GNNModel(d,h,c)
model = model.to(device)
max_accuracy = 0.5156
path = "class/models/GNN.4.1.pt"
model.load_state_dict(torch.load(path))
path = "class/models/GNN.4.1.pt"
tv_run(epochs, model, lr, alpha, max_accuracy, path, verbose = 2)

<ipython-input-30-280edc2dd513>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))



 --------- 
Epoch: 1

Epoch 1 train loss: 1.6146
Epoch 1 train accuracy: 0.5366
Epoch 1 dev loss: 1.7683
Epoch 1 dev accuracy: 0.5031

 --------- 
Epoch: 2

Epoch 2 train loss: 1.6030
Epoch 2 train accuracy: 0.5387
Epoch 2 dev loss: 1.7540
Epoch 2 dev accuracy: 0.5073

 --------- 
Epoch: 3

Epoch 3 train loss: 1.5757
Epoch 3 train accuracy: 0.5441
Epoch 3 dev loss: 1.7486
Epoch 3 dev accuracy: 0.5093

 --------- 
Epoch: 4

Epoch 4 train loss: 1.5531
Epoch 4 train accuracy: 0.5550
Epoch 4 dev loss: 1.7675
Epoch 4 dev accuracy: 0.4761

 --------- 
Epoch: 5

Epoch 5 train loss: 1.5441
Epoch 5 train accuracy: 0.5548
Epoch 5 dev loss: 1.7297
Epoch 5 dev accuracy: 0.4969

 --------- 
Epoch: 6

Epoch 6 train loss: 1.5326
Epoch 6 train accuracy: 0.5573
Epoch 6 dev loss: 1.7417
Epoch 6 dev accuracy: 0.5000

 --------- 
Epoch: 7

Epoch 7 train loss: 1.5161
Epoch 7 train accuracy: 0.5588
Epoch 7 dev loss: 1.7574
Epoch 7 dev accuracy: 0.4834

 --------- 
Epoch: 8

Epoch 8 train loss: 1.5222
Epoch

Stats(train_loss=1.3559853447244523, train_accuracy=0.6029397796352584, dev_loss=1.6307920217514038, dev_accuracy=0.549792531120332, epoch=21, lr=0.000124, alpha=4.6e-05, max_accuracy=0.549792531120332)

# change batch size
# sparse metrices
# mixed precision training
# memory pinning